# World of Quantum Hands-on: QisK-I-T

This Notebook is divided into 2 parts:
- Code for error suppresion & dynamic decopuling Research Question (RQ1)
- Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

    - Data-Preparation & Pass Selection and Experimentation
    - Model-Training and Experimentation
    - Evaluation

## Technical: Before you begin
- `conda create -n [env_name]`
- `conda activate [env_name]`
- `pip install -r ./requirements.txt`
- `conda update -all`

## Imports

In [182]:
import ast
import numpy as np
import pandas as pd
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# 1. Code for error suppresion & dynamic decopuling Research Question (RQ1)

In [183]:
# PLACEHOLDER RINORS AND JOSHUAS CODE

# 2. Code for automated supervised learning best configuration Prediction in Optimization stage (RQ2)

Below is the visualized pipeline

![My diagram](Resources/graph.png)

## Data-Preparation & Pass Selection and Experimentation

### hier der hackaton.py etc.

## Model-Training and Experimentation

### Data preprocessing

We do following steps:

1. a
2. b
3. c

In [184]:
dataset = pd.read_csv('data/training_data.csv', header=None) # TODO: remove header=NONE in next csv version

dataset.rename(
    columns={
      dataset.columns[0]:  "ID",
      dataset.columns[-1]: "configuration"
    },
    inplace=True
)

dataset.head()


,ID,1,2,3,4,5,6,7,8,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [185]:
df_configurations_with_id_with_duplicates = dataset[['ID', 'configuration']].copy()
df_features_with_id = dataset.drop(['configuration'], axis=1).copy()

In [186]:
df_configurations_with_id_with_duplicates.head()

,ID,configuration
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [187]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0


In [188]:
df_configurations_with_id_with_duplicates['config_arr'] = (
    df_configurations_with_id_with_duplicates['configuration']
      .apply(ast.literal_eval)          # "[1,0,1,...]" → [1,0,1,...]
      #.apply(lambda lst: np.array(lst, dtype=int))
)
#df_ored_configuration = (
#    df_configurations_with_id_with_duplicates
#      .groupby('ID')['config_arr']
#      .agg(lambda arrs: np.bitwise_or.reduce(arrs.tolist()))
#      .reset_index()
#      .rename(columns={'config_arr':'label_vec'})
#)

df_configurations = df_configurations_with_id_with_duplicates[['ID','config_arr']].copy()

In [189]:
df_configurations.head()

,ID,config_arr
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [190]:
df_features_with_id.head()

,ID,1,2,3,4,5,6,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,0,90,0


In [191]:
single_value_cols = df_features_with_id.columns[df_features_with_id.nunique(dropna=False) == 1]
single_value_cols

Index([6], dtype='object')

In [192]:
df_feat = df_features_with_id.drop(single_value_cols, axis=1).copy()
df_feat.head()

,ID,1,2,3,4,5,7,8
0,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
1,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
2,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
3,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0
4,ae_nativegates_ibm_qiskit_opt0_10.qasm,10,183,20,375,90,90,0


In [193]:
df_feat.drop(['ID'], inplace=True, axis=1)
df_configurations.drop(['ID'], inplace=True, axis=1)

In [194]:
df_feat

,1,2,3,4,5,7,8
0,10,183,20,375,90,90,0
1,10,183,20,375,90,90,0
2,10,183,20,375,90,90,0
3,10,183,20,375,90,90,0
4,10,183,20,375,90,90,0
...,...,...,...,...,...,...,...
2165,9,54,18,297,108,108,0
2166,9,54,18,297,108,108,0
2167,9,54,18,297,108,108,0
2168,9,54,18,297,108,108,0


In [195]:
df_configurations

,config_arr
0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ..."
1,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...
2165,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2166,"[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2167,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2168,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [196]:
df_feat.dtypes

1    int64
2    int64
3    int64
4    int64
5    int64
7    int64
8    int64
dtype: object

In [197]:
# TODO: add scaling, log transform? Not quite enough time to understand the dataset.

y = np.vstack(df_configurations['config_arr'].values)
X = df_feat.values

### Configuration for cross-validation of hyperparameters and of the model to gain reliable insights

In [198]:
# Configuration
DO_HPO          = True
HPO_NUM_TRIALS  = 40
N_OUTER_FOLDS   = 10
N_INNER_FOLDS   = 5
RANDOM_STATE    = 43

BASELINE_PARAMS = {
    "n_estimators": 100,
}

In [199]:
from sklearn.metrics import make_scorer, hamming_loss

base_clf = XGBClassifier(
    **BASELINE_PARAMS,
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    tree_method='hist',
    base_score=0.5,
    eval_metric='logloss'
)
multiLabelClassifier = MultiOutputClassifier(base_clf, n_jobs=-1)

outer_cv = MultilabelStratifiedKFold(
    n_splits=N_OUTER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

inner_cv = MultilabelStratifiedKFold(
    n_splits=N_INNER_FOLDS,
    shuffle=True,
    random_state=RANDOM_STATE,
)

scoring = {
    "f1_micro": "f1_micro",
    "f1_macro": "f1_macro",
    "hamming": make_scorer(hamming_loss, greater_is_better=False)
}

cv_results = cross_validate(
    multiLabelClassifier, X, y,
    cv=outer_cv,
    scoring=scoring,
    return_train_score=False,
    n_jobs=-1
)
# TODO: feature importance study
print("Outer-fold scores:")
for name in scoring:
    mean = cv_results[f"test_{name}"].mean()
    std  = cv_results[f"test_{name}"].std()
    print(f"  {name:>10s}: {mean:7.3f} ± {std:.3f}")

/opt/anaconda3/envs/Notebook/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/Notebook/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/Notebook/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metri

Outer-fold scores:
    f1_micro:   0.678 ± 0.012
    f1_macro:   0.251 ± 0.006
     hamming:  -0.078 ± 0.003


/opt/anaconda3/envs/Notebook/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [200]:
multiLabelClassifier.fit(X, y)

,estimator,"XGBClassifier...ree=None, ...)"
,n_jobs,-1
,objective,'binary:logistic'
,base_score,0.5
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None


## Evaluation
We use the trained model to predict the optimal configuration for new circuits.
Then, we evaluate performance based on the number of qubits in the transpiled output, comparing it against Qiskit's default PassManager strategies: fast, normal, and slow.

In [201]:
# TODO: Add evaluation code against default optimization routine